In [11]:
from __future__ import annotations

import os
from typing import Optional, Dict, List
from dotenv import load_dotenv
from google.cloud import bigquery

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = GOOGLE_APPLICATION_CREDENTIALS

In [14]:
client = bigquery.Client()

In [ ]:
PROJECT_ID = "automacoes-481202" 
DATASET_ID = "case_telecom" 

CSV_DELIMITER = "," 

SKIP_LEADING_ROWS = 1

FILES_TO_TABLES: List[Dict[str, str]] = [
    {
        "path": "../output/base_clientes_tratada.csv",
        "table": "base_clientes",
    },
    {
        "path": "../output/base_produtos_tratada.csv",
        "table": "base_produtos",
    },
    {
        "path": "../output/base_vendas_tratada.csv",
        "table": "base_vendas",
    },
    {
        "path": "../output/base_info_vendas_tratada.csv",
        "table": "informacoes_vendas",
    },
]

# Comportamento de escrita:
# - WRITE_TRUNCATE: substitui a tabela toda a cada carga
# - WRITE_APPEND: acrescenta registros (use com cuidado)
WRITE_DISPOSITION = bigquery.WriteDisposition.WRITE_TRUNCATE

CREATE_DISPOSITION = bigquery.CreateDisposition.CREATE_IF_NEEDED


def get_bq_client(project_id: str) -> bigquery.Client:

    return bigquery.Client(project=project_id)


def ensure_dataset(client: bigquery.Client, project_id: str, dataset_id: str, location: str = "US") -> None:

    dataset_ref = bigquery.Dataset(f"{project_id}.{dataset_id}")
    dataset_ref.location = location

    try:
        client.get_dataset(dataset_ref)
        print(f"[OK] Dataset já existe: {project_id}.{dataset_id}")
    except Exception:
        client.create_dataset(dataset_ref, exists_ok=True)
        print(f"[CRIADO] Dataset criado: {project_id}.{dataset_id} (location={location})")


def load_csv_to_table(
    client: bigquery.Client,
    project_id: str,
    dataset_id: str,
    table_name: str,
    file_path: str,
    field_delimiter: str = ";",
    skip_leading_rows: int = 1,
    write_disposition: str = bigquery.WriteDisposition.WRITE_TRUNCATE,
    create_disposition: str = bigquery.CreateDisposition.CREATE_IF_NEEDED,
    autodetect_schema: bool = True,
) -> None:

    table_id = f"{project_id}.{dataset_id}.{table_name}"

    job_config = bigquery.LoadJobConfig(
        source_format=bigquery.SourceFormat.CSV,
        field_delimiter=field_delimiter,
        skip_leading_rows=skip_leading_rows,
        autodetect=autodetect_schema,
        write_disposition=write_disposition,
        create_disposition=create_disposition,
        allow_quoted_newlines=True,  # tolerância a quebras de linha em campos com aspas
    )

    with open(file_path, "rb") as f:
        load_job = client.load_table_from_file(
            f,
            destination=table_id,
            job_config=job_config,
        )

    print(f"[ENVIANDO] {file_path} -> {table_id}")
    load_job.result()  # espera finalizar

    table = client.get_table(table_id)
    print(f"[OK] Carregado: {table_id} | linhas: {table.num_rows} | colunas: {len(table.schema)}")


def main() -> None:
    client = get_bq_client(PROJECT_ID)

    # 2) Dataset (cria se não existir)
    # Ajuste location se necessário.
    ensure_dataset(client, PROJECT_ID, DATASET_ID, location="US")

    # 3) Cargas (1 arquivo = 1 tabela)
    for item in FILES_TO_TABLES:
        file_path = item["path"]
        table_name = item["table"]

        if not os.path.exists(file_path):
            print(f"[ERRO] Arquivo não encontrado: {file_path}")
            continue

        load_csv_to_table(
            client=client,
            project_id=PROJECT_ID,
            dataset_id=DATASET_ID,
            table_name=table_name,
            file_path=file_path,
            field_delimiter=CSV_DELIMITER,
            skip_leading_rows=SKIP_LEADING_ROWS,
            write_disposition=WRITE_DISPOSITION,
            create_disposition=CREATE_DISPOSITION,
            autodetect_schema=True,
        )


if __name__ == "__main__":
    main()


[OK] Dataset já existe: automacoes-481202.case_telecom
[ENVIANDO] ../output/base_clientes_tratada.csv -> automacoes-481202.case_telecom.base_clientes
[OK] Carregado: automacoes-481202.case_telecom.base_clientes | linhas: 10000 | colunas: 15
[ENVIANDO] ../output/base_produtos_tratada.csv -> automacoes-481202.case_telecom.base_produtos
[OK] Carregado: automacoes-481202.case_telecom.base_produtos | linhas: 6 | colunas: 2
[ENVIANDO] ../output/base_vendas_tratada.csv -> automacoes-481202.case_telecom.base_vendas
[OK] Carregado: automacoes-481202.case_telecom.base_vendas | linhas: 5000 | colunas: 8
[ENVIANDO] ../output/base_info_vendas_tratada.csv -> automacoes-481202.case_telecom.informacoes_vendas
[OK] Carregado: automacoes-481202.case_telecom.informacoes_vendas | linhas: 5000 | colunas: 11
